# Set-up

In [ ]:
pip install kaggle

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import RepeatedKFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import RFE
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

# Import Data from Kaggle API

In [5]:
raw_data = pd.read_csv('Listings.csv',encoding='ISO-8859-1' ,dtype={5: str, 13: str})

In [6]:
raw_data.head(5)

,listing_id,name,host_id,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,...,minimum_nights,maximum_nights,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable
0,281420,"Beautiful Flat in le Village Montmartre, Paris",1466919,2011-12-03,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
1,3705183,39 mÃÂ² Paris (Sacre CÃ âur),10328771,2013-11-29,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
2,4082273,"Lovely apartment with Terrace, 60m2",19252768,2014-07-31,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
3,4797344,Cosy studio (close to Eiffel tower),10668311,2013-12-17,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
4,4823489,Close to Eiffel Tower - Beautiful flat : 2 rooms,24837558,2014-12-14,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f


# Data Analysis, Data Clean and Data Preprocessing

In [4]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279712 entries, 0 to 279711
Data columns (total 33 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   listing_id                   279712 non-null  int64  
 1   name                         279539 non-null  object 
 2   host_id                      279712 non-null  int64  
 3   host_since                   279547 non-null  object 
 4   host_location                278872 non-null  object 
 5   host_response_time           150930 non-null  object 
 6   host_response_rate           150930 non-null  float64
 7   host_acceptance_rate         166625 non-null  float64
 8   host_is_superhost            279547 non-null  object 
 9   host_total_listings_count    279547 non-null  float64
 10  host_has_profile_pic         279547 non-null  object 
 11  host_identity_verified       279547 non-null  object 
 12  neighbourhood                279712 non-null  object 
 13 

## 考虑设置价格作为因变量，将其他特征作为协变量，使用机器学习算法根据特征对价格进行预测

## 删除无用列

In [19]:
raw_data['district'].value_counts()
# 由于该变量缺失值过多，难以进行补充，并且对预测效果帮助不大，因此删去

Manhattan        16545
Brooklyn         14468
Queens            4700
Bronx              992
Staten Island      289
Name: district, dtype: int64

In [21]:
#id对价格不会产生影响，同样删除，
drop_var = ['listing_id', 'host_id', 'district']
raw_data.drop(columns=drop_var, inplace = True)

In [22]:
raw_data.head(5)
#列数由33变为30

,name,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,...,minimum_nights,maximum_nights,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable
0,"Beautiful Flat in le Village Montmartre, Paris",2011-12-03,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,t,f,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
1,39 mÃÂ² Paris (Sacre CÃ âur),2013-11-29,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,t,t,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
2,"Lovely apartment with Terrace, 60m2",2014-07-31,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,t,f,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
3,Cosy studio (close to Eiffel tower),2013-12-17,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,t,t,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
4,Close to Eiffel Tower - Beautiful flat : 2 rooms,2014-12-14,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,t,f,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f


## 剔除缺失值与类型转化

In [18]:
#得到将转化的二值变量
dummy_var=['host_is_superhost','host_has_profile_pic','host_identity_verified','instant_bookable']
for element in dummy_var:
    raw_data=raw_data[raw_data[element].isnull() == False]
#剔除缺失值

array(['f', 't', nan], dtype=object)

In [34]:
#进行类型转化
for element in dummy_var:    
    #raw_data2[element] = raw_data[element].astype(str)
    raw_data[element] = raw_data[element].map({'t':1,'f':0})
    raw_data[element].astype('int32')    

In [35]:
for element in dummy_var:  
    print(raw_data[element].unique())
#剔除成功

[0 1]
[1 0]
[0 1]
[0 1]


In [36]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 279547 entries, 0 to 279711
Data columns (total 30 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   name                         279375 non-null  object 
 1   host_since                   279547 non-null  object 
 2   host_location                278872 non-null  object 
 3   host_response_time           150930 non-null  object 
 4   host_response_rate           150930 non-null  float64
 5   host_acceptance_rate         166625 non-null  float64
 6   host_is_superhost            279547 non-null  int64  
 7   host_total_listings_count    279547 non-null  float64
 8   host_has_profile_pic         279547 non-null  int64  
 9   host_identity_verified       279547 non-null  int64  
 10  neighbourhood                279547 non-null  object 
 11  city                         279547 non-null  object 
 12  latitude                     279547 non-null  float64
 13 

In [39]:
#  根据汇率,将价格统一转换为人民币计价
ex_rate = {"Paris": 7.57, 
           "New York": 6.84,
           "Sydney": 4.63, 
           "Rome": 7.57,
           "Rio de Janeiro": 1.38,
           "Istanbul": 0.35,
           "Mexico City": 0.37,
           "Bangkok": 0.20,
           "Cape Town": 0.38, 
           "Hong Kong": 0.87}
city_copy = raw_data['city']
raw_data['price_CNY'] = city_copy.map(ex_rate) * raw_data['price']

In [40]:
# 检查是否有无重复值
raw_data[raw_data.duplicated() == True]

,name,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,...,maximum_nights,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,price_CNY
66632,Moderno depto con excelentes amenidades,2020-09-21,"Mexico City, Mexico City, Mexico",within an hour,1.0,0.93,1,5.0,1,1,...,1125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,303.40
103516,Triple room nicely located beside Sultanahmet,2017-01-10,TR,within an hour,1.0,1.00,0,9.0,1,1,...,1125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,120.75
103532,Family room nicely located beside Sultanahmet,2017-01-10,TR,within an hour,1.0,1.00,0,9.0,1,1,...,1125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,136.85
103533,Triple room nicely located beside Sultanahmet,2017-01-10,TR,within an hour,1.0,1.00,0,9.0,1,1,...,1125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,120.75
104081,Double room nicely located beside Sultanahmet,2017-01-10,TR,within an hour,1.0,1.00,0,9.0,1,1,...,1125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,103.60
142106,Jaba Suit BeÃ Å¸iktaÃ Å¸'ta 25mÃÂ² Konforlu ...,2020-10-21,"BeÃ Å¸iktaÃ Å¸, Istanbul, Turkey",within an hour,1.0,1.00,0,0.0,1,1,...,1125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,70.00
142108,Jaba Suit BeÃ Å¸iktaÃ Å¸'ta 25mÃÂ² Konforlu ...,2020-10-21,"BeÃ Å¸iktaÃ Å¸, Istanbul, Turkey",within an hour,1.0,1.00,0,0.0,1,1,...,1125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,70.00
150465,Regolo Rooms Vaticano - Deluxe Double Room,2019-08-22,IT,NaN,NaN,NaN,0,7.0,1,1,...,1125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,999.24
150467,Regolo Rooms Vaticano - Twin Room,2019-08-22,IT,NaN,NaN,NaN,0,7.0,1,1,...,1125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,999.24
150548,"Room close from the ""Champs Ãâ°lysees""",2020-03-10,"Paris, Ile-de-France, France",NaN,NaN,NaN,0,4.0,1,1,...,1125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2271.00


In [42]:
# 删除重复数据
raw_data.drop_duplicates(keep=False, inplace=True)

In [46]:
#根据z-score去除极端值
z_score=(raw_data['price_CNY']-raw_data['price_CNY'].mean())/raw_data['price_CNY'].std()
outlier=z_score.abs() > 1.96
raw_data=raw_data[outlier == False]
data=raw_data

## 数据清理完成，进行数据可视化

In [ ]:
# 将数据分为训练集与测试集
X=
Y=data['price_CNY']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=111)
display(ex_rate[city])

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
airbnb_tr.hist(bins = 20, figsize=(20,15))
plt.style.use('seaborn-whitegrid')
plt.show()

# Linear Models

In [ ]:
airbnb_tr['price'].describe()

In [ ]:
Linear

In [ ]:
Lassco

In [ ]:
Ridge

In [ ]:
ElasticNet

###  6 folds cross validation model performance